# Sesión de Trabajo 5: PREPARACIÓN DE LA BASE DE DATOS PARA ENTRENAR MODELOS PREDICTIVOS
**Asignatura:** Ciència de Dades i Intel·ligència Artificial Aplicades a la Construcció i Estructures  
**Institución:** ETSEIB - UPC  

---

## Objetivos de la sesión
El entrenamiento de modelos de aprendizaje autónomo involucra una serie de requisitos adicionales a nuestra base de datos “limpia” que conseguimos al finalizar la Sesión de Trabajo 3. La sesión de trabajo actual se centra en llevar a cabo dichas modificaciones.


* **S5.T1.** Selección inicial de variables de entrada y salida del modelo

* **S5.T2.** Escalado/Normalización de las variables continuas

* **S5.T3.** Codificación de las variables categóricas

---



### S5.T0. Importar la base de datos
* Conectamos con Google Drive.
* Cargamos el archivo `BBDD_ST3.csv`.
* Optimizamos los tipos de datos (`float32` e `int32`) para no saturar la RAM de Colab.
* Forzamos columnas categóricas o identificadores (como códigos postales) a string para evitar cálculos matemáticos erróneos.

⏳ *Tiempo de ejecución estimado: 1-2 minutos (dependiendo de la velocidad de conexión a Drive).*


In [ ]:

# Importar librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler

# Montar Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Ruta y Carga
ruta_drive = '/content/drive/MyDrive/Sostenibilidad/BBDD_ST3.csv'
df = pd.read_csv(ruta_drive, low_memory=False)

# Optimización
df["CODI_POSTAL"] = df["CODI_POSTAL"].astype('str')
df["VALOR AILLAMENTS CTE"] = df["VALOR AILLAMENTS CTE"].astype('str')
df["VALOR FINESTRES CTE"] = df["VALOR FINESTRES CTE"].astype('str')
float64_cols = df.select_dtypes(include='float64').columns.tolist()
df[float64_cols] = df[float64_cols].astype('float32')
int64_cols = df.select_dtypes(include='int64').columns.tolist()
df[int64_cols] = df[int64_cols].astype('int32')

print("Base de datos limpia cargada y optimizada.")

---

### S5.T1. Selección inicial de variables de entrada y salida del modelo

A pesar de que ahora ya disponéis de una base de datos “limpia” que os ha permito hacer correlaciones y extraer conocimiento de la sostenibilidad del parque edificado catalán, aún no habéis seleccionado las variables de entrada y la variable de salida que usaréis en vuestros modelos. Pensad que este puede ser un proceso iterativo, por lo que inicialmente se recomienda adoptar unenfoque simplificado para facilitar el entrenamiento en tiempos razonables, y en base a los resultados, ir ajustando o añadiendo parámetros.

* **S5.T1.1.**  Verifica que tu dataframe no contiene NaNs

* **S5.T1.2.**  Selección de la variable de salida (output)

* **S5.T1.3.**  Selección de la variables de entrada (inputs)

---


* **S5.T1.1.** Verifica que tu dataframe no contiene NaNs

>La mayoría de procesos de entrenamiento de modelos de aprendizaje autonomo no pueden operar con NaNs en la base de datos.

>Para comprobar que no hay NaNs en tu dataframe, puedes usar:

>>> `df.info()` o `df.isnull().sum()`

>Para eliminar los NaNs, podéis referiros a la ST3


In [ ]:
#Insertar código aquí

* **S5.T1.2.** Selección de la variable de salida

>Se recomienda elegir la misma que en la sesión de trabajo 4: `Qualificació de consum d'energia primaria no renovable` o `Qualificacio d'emissions de CO2`). .

>Elimina todos los demás outputs. Puedes usar:

>>> `df= df.drop("Energia primària no renovable", axis=1)`


In [ ]:
#Insertar código aquí

* **S5.T1.3.** Selección de la variables de entrada (inputs)

>Elimina aquellas variables de entrada categóricas que presenten un núm. de campos > 1000. En caso de duda, mejor mantener la variable.

>Ejemplo:

>>> `df= df.drop("CODI_POSTAL", axis=1)`

In [ ]:
#Insertar código aquí


---

### S5.T2. Escalado/Normalización de las variables continuas

Las variables numéricas (como metros cuadrados) deben estar en un rango similar para que el modelo no dé más peso a una variable solo por tener números más grandes.

* **S5.T2.1.**  Realiza el escalado/normalización a aquellas variables continuas que consideres oportunas.


---

* **S5.T2.1.** Realiza el escalado/normalización a aquellas variables continuas que consideres oportunas

>>**-Alternativa 1: MinMaxScaler**. Este escalador transforma los datos al rango [0, 1]. Es ideal cuando los datos están acotados y no tienen valores extremadamente altos o bajos (outliers).

>>**-Alternativa 2: RobustScaler**. Este escalador utiliza la mediana y los cuartiles. Es resistente a los valores atípicos (outliers).


In [ ]:
# EJEMPLO ALTERNATIVA 1: MinMaxScaler en METRES_CADASTRE
# 1. Preparar la figura con 2 subgráficos (1 fila, 2 columnas)
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# --- GRÁFICO 1: ANTES DEL ESCALADO ---
# Usamos histplot con kde=True
sns.histplot(df['METRES_CADASTRE'], kde=True, ax=axes[0], color='skyblue')
axes[0].set_title("Distribución Original")
axes[0].set_xlabel("Metros Cuadrados")

# --- PROCESO DE ESCALADO ---
scaler_minmax = MinMaxScaler()
# OJO: Hacemos el fit_transform en una copia o variable auxiliar
# para no sobreescribir el df original antes de graficar el "Después"
df['METRES_ESCALADOS'] = scaler_minmax.fit_transform(df[['METRES_CADASTRE']])

# --- GRÁFICO 2: DESPUÉS DEL ESCALADO ---
sns.histplot(df['METRES_ESCALADOS'], kde=True, ax=axes[1], color='salmon')
axes[1].set_title("Distribución tras MinMaxScaler (Después)")
axes[1].set_xlabel("Valor Escalado (0 a 1)")

# Ajustar espacio entre gráficos para que no se solapen las etiquetas
plt.tight_layout()
plt.show()

# Limpieza final: borrar la columna auxiliar y modifica la original
df['METRES_CADASTRE'] = df['METRES_ESCALADOS']
df.drop(columns=['METRES_ESCALADOS'], inplace=True)

In [ ]:
# EJEMPLO ALTERNATIVA 2: RobustScaler en METRES_CADASTRE
# 1. Preparar la figura con 2 subgráficos (1 fila, 2 columnas)
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# --- GRÁFICO 1: ANTES DEL ESCALADO ---
# Usamos histplot con kde=True
sns.histplot(df['METRES_CADASTRE'], kde=True, ax=axes[0], color='skyblue')
axes[0].set_title("Distribución Original")
axes[0].set_xlabel("Metros Cuadrados")

# --- PROCESO DE ESCALADO ---
scaler_robust = RobustScaler()
# OJO: Hacemos el fit_transform en una copia o variable auxiliar
# para no sobreescribir el df original antes de graficar el "Después"
df['METRES_ESCALADOS'] = scaler_robust.fit_transform(df[['METRES_CADASTRE']])

# --- GRÁFICO 2: DESPUÉS DEL ESCALADO ---
sns.histplot(df['METRES_ESCALADOS'], kde=True, ax=axes[1], color='salmon')
axes[1].set_title("Distribución tras RobustScaler (Después)")
axes[1].set_xlabel("Valor Escalado")

# Ajustar espacio entre gráficos para que no se solapen las etiquetas
plt.tight_layout()
plt.show()

# Limpieza final: borrar la columna auxiliar y modifica la original
df['METRES_CADASTRE'] = df['METRES_ESCALADOS']
df.drop(columns=['METRES_ESCALADOS'], inplace=True)

>TAREA: Normaliza/Escala las demás variables continuas con MinMaxScaler o RobustScaler.

>Escribe el código a continuación:

In [ ]:
#Insertar código aquí para la normalización de la variable continua 1

In [ ]:
#Insertar código aquí para la normalización de la variable continua 2 ...


---

### S5.T3. Codificación de las variables categóricas

Los modelos de aprendizaje autónomo no pueden trabajar directamente con datos categóricos, necesitan datos numéricos.Para ello, antes de entrenar o evaluar cualquier modelo es esencial que codifiquéis las variables categóricas de vuestra basededatos. Se distinguen principalmente tres enfoques distintos para llevar a cabo esta tarea en función de la naturaleza de la variable categórica.

* **S5.T3.1. Variables categóricas binarias:**  Únicamente 2 campos posibles (Sí/No, ...).

* **S5.T3.2. Variables categóricas ordinales:**  Tienen un orden o jerarquía.   

* **S5.T3.3. Variables categóricas nominales:**  Sin orden jerárquico.


---



---

TAREA INICIAL: Haz doble-click en esta celda y completa las listas a continuación con los nombres de las columnas de tu dataframe según el tipo de variable categórica:

* **Variables Binarias:** `["VEHICLE ELECTRIC", ...]`
* **Variables Ordinales:** `["Qualificació...", ...]`
* **Variables Nominales:** `["Eina de certificacio", ...]`

---

* **S5.T3.1.** Variables categóricas binarias

>Únicamente tienen dos valores posibles. En este caso, simplemente hay que reemplazar los valores originales nominales por valores numéricos (0/1). Recordad que solo deben modificarse aquellas que no estén en formato numérico (Si/No).



In [ ]:
# Ejemplo Variable Binaria: "VEHICLE ELECTRIC"

# Mostrar el estado inicial para la clase
print("--- ESTADO INICIAL (Primeros 5 registros) ---")
print(df[["VEHICLE ELECTRIC"]].head())

# 3. Aplicar la codificación numérica (Sustitución directa)
# Asignamos 0 a la ausencia (NO) y 1 a la presencia (SI)
mapeo_binario = {'NO': 0, 'SI': 1}
df["VEHICLE ELECTRIC"] = df["VEHICLE ELECTRIC"].replace(mapeo_binario)

# 4. Mostrar el resultado final
print("\n--- ESTADO TRAS LA CODIFICACIÓN ---")
print(df[["VEHICLE ELECTRIC"]].head())

# 5. Verificación de tipo de dato
print(f"\nNuevo tipo de dato de la columna: {df['VEHICLE ELECTRIC'].dtype}")

>TAREA: Completa la codificación de las demás variables binarias.

>Escribe el código a continuación (puedes ponerlo todo en una única celda, como prefieras):

In [ ]:
#Insertar código aquí para la codificación de las demás variables binarias.

* **S5.T3.2.** Variables categóricas ordinales

>Tienen un orden lógico o jerarquía (ej. Calificaciones energéticas de A a G).


In [ ]:
# Ejemplo Variable Ordinal: "Qualificació de consum d'energia primaria no renovable"

# 1. Mostrar los valores únicos antes de la transformación
# Es importante que vean que las categorías tienen un orden lógico (A-G)
print("--- CATEGORÍAS ORIGINALES ---")
print(df["Qualificació de consum d'energia primaria no renovable"].unique())

# 2. Definir el diccionario de mapeo jerárquico
# Asignamos valores numéricos que respeten el orden de eficiencia
# G (peor) = 0, A (mejor) = 6
ord_energetico = {
    'G': 0,
    'F': 1,
    'E': 2,
    'D': 3,
    'C': 4,
    'B': 5,
    'A': 6
}

# 3. Aplicar la transformación mediante .map()
# Usamos .map() porque es más eficiente para sustituciones completas de diccionarios
df["Qualificació de consum d'energia primaria no renovable"] = \
    df["Qualificació de consum d'energia primaria no renovable"].map(ord_energetico)

# 4. Mostrar el resultado final y la distribución
print("\n--- RESULTADO TRAS EL MAPEO ORDINAL ---")
print(df["Qualificació de consum d'energia primaria no renovable"].value_counts().sort_index())


>TAREA: Completa la codificación de las demás variables ordinales.

>Escribe el código a continuación (puedes ponerlo todo en una única celda, como prefieras):

In [ ]:
#Insertar código aquí para la codificación de las demás variables ordinales.

* **S5.T3.2.** Variables categóricas nominales

>Aquí no hay un orden (la herramienta "CE3X" no es "mayor" que "HULC"), por lo que no podemos usar números seguidos como 0, 1, 2. Si lo hiciéramos, el modelo pensaría erróneamente que existe una jerarquía.

Utilizamos One-Hot Encoding (get_dummies), que expande la columna en varias columnas de "presencia/ausencia" (1 o 0).

In [ ]:
# Ejemplo Variable Ordinal: "Eina de certificacio"

# 1. Mostrar estado inicial
print("--- CATEGORÍAS NOMINALES DETECTADAS ---")
categorias = df["Eina de certificacio"].unique()
print(categorias)

# 2. Aplicar One-Hot Encoding (pd.get_dummies)
# Esto eliminará la columna original y creará una por cada categoría
df = pd.get_dummies(df, columns=["Eina de certificacio"])

# 3. Mostrar el resultado de las nuevas columnas creadas
# Buscamos las columnas que empiezan por el nombre original para ver el cambio
nuevas_cols = [col for col in df.columns if "Eina de certificacio" in col]

print("\n--- NUEVAS COLUMNAS CREADAS (Dummies) ---")
print(df[nuevas_cols].head())

# 4. Verificación del tamaño del DataFrame
# Al usar dummies, el número de columnas (ancho del df) siempre aumenta
print(f"\nNúmero total de columnas ahora: {df.shape[1]}")

>TAREA: Completa la codificación de las demás variables nominales.

>Escribe el código a continuación (puedes ponerlo todo en una única celda, como prefieras):

In [ ]:
#Insertar código aquí para la codificación de las demás variables nominales.

Eliminar observaciones duplicadas.

In [ ]:
n_antes_dupl = len(df)
df = df.drop_duplicates()
print(f"Registros duplicados eliminados: {n_antes_dupl - len(df)}")

Guardar BBDD limpia codificada

In [ ]:
# Usamos la misma carpeta donde cargamos la BBDD_ST3
ruta_salida = '/content/drive/MyDrive/Sostenibilidad/BBDD_ST5.csv'
df.to_csv(ruta_salida, index=False)